requests 위치 데이터 가져오기
json_normalize(): JSON 데이터 DataFrame으로 전환
folium 을 사용하여 지도에 위치 표시

In [1]:
import requests
import json
import pandas as pd
#from pandas.io.json import json_normalize

In [2]:
data = {
    'ins_lat':'37.56682', # 지정한 위도와 경도에서 가까운 순으로 나열
    'ins_lng':'126.97865',
    'p_sido_cd':'01', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
    
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)

r.text[:1000] # 수신된 데이터의 앞부분만 확인

'{"list":[{"seq":0,"sido_cd":null,"sido_nm":null,"gugun_cd":null,"gugun_nm":null,"code_order":null,"view_yn":null,"store_num":null,"sido":null,"gugun":null,"address":null,"new_img_nm":null,"p_pro_seq":0,"p_view_yn":null,"p_sido_cd":"","p_gugun_cd":"","p_store_nm":null,"p_theme_cd":null,"p_wireless_yn":null,"p_smoking_yn":null,"p_book_yn":null,"p_music_yn":null,"p_terrace_yn":null,"p_table_yn":null,"p_takeout_yn":null,"p_parking_yn":null,"p_dollar_assent":null,"p_card_recharge":null,"p_subway_yn":null,"stb_store_file_renew":null,"stb_store_theme_renew":null,"stb_store_time_renew":null,"stb_store_lsm":null,"s_code":"1509","s_name":"역삼아레나빌딩","tel":"02-568-3762","fax":"02-568-3763","sido_code":"01","sido_name":"서울","gugun_code":"0101","gugun_name":"강남구","addr":"서울특별시 강남구 역삼동 721-13 아레나빌딩","park_info":null,"new_state":null,"theme_state":"T04@T05@T08@T16@T17@T20@T21@P80@P90","new_bool":0,"search_text":"","ins_lat":"","ins_lng":"","in_distance":0,"out_distance":"9.26","all_search_cnt":-1,"add

In [3]:
jo = json.loads(r.text)
df = pd.json_normalize(jo, 'list')


In [4]:
# 행(row)수, 서울 457개 매장
len(df) 

509

In [5]:
# 컬럼수 111개
df.columns

Index(['seq', 'sido_cd', 'sido_nm', 'gugun_cd', 'gugun_nm', 'code_order',
       'view_yn', 'store_num', 'sido', 'gugun',
       ...
       'p60', 'p30', 'p70', 'p40', 'p80', 't22', 't21', 'p90', 't05', 't30'],
      dtype='object', length=122)

In [6]:
df = df[['s_name', 'lat', 'lot', 'sido_name', 'gugun_name', 'doro_address', 'tel']]
df.head()

,s_name,lat,lot,sido_name,gugun_name,doro_address,tel
0,역삼아레나빌딩,37.501087,127.043069,서울,강남구,서울특별시 강남구 언주로 425 (역삼동),02-568-3762
1,논현역사거리,37.510178,127.022223,서울,강남구,서울특별시 강남구 강남대로 538 (논현동),02-3442-3672
2,신사역성일빌딩,37.514132,127.020563,서울,강남구,서울특별시 강남구 강남대로 584 (논현동),02-547-3858
3,국기원사거리,37.499517,127.031495,서울,강남구,서울특별시 강남구 테헤란로 125 (역삼동),02-568-3668
4,스탈릿대치R,37.494668,127.062583,서울,강남구,서울특별시 강남구 남부순환로 2947 (대치동),02-568-3704


In [7]:
import folium

# 서울 시청 
map_osm = folium.Map(location=(37.56629, 126.979808))
map_osm

In [8]:
# 확대 지정 (zoom_start)
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17)
map_osm

In [9]:
# 지도 tiles 지정
map_osm = folium.Map(location=(37.56629, 126.979808), zoom_start=17, tiles='Stamen Toner')
map_osm

In [10]:
시청_좌표=(37.56629, 126.979808)
 
map_osm = folium.Map(location=시청_좌표, zoom_start=17)
folium.Marker(시청_좌표, popup='시청').add_to(map_osm)
map_osm

In [11]:
# 천안 시청 
천안시청=(36.8151, 127.1139)
data = {
    'ins_lat': '36.8151', # 천안 시청의 좌표
    'ins_lng': '127.1139',
    'p_sido_cd':'14', # 01=서울시, 08=경기 ... 16=제주
    'p_gugun_cd':'',  # 세부지역 (지정하지 않으면 시/도 전체)
    'in_biz_cd':'',
    'set_date':'',
    'iend':'1000',
}
url = 'https://www.istarbucks.co.kr/store/getStore.do'
r = requests.post(url, data=data)
df = pd.json_normalize(json.loads(r.text), 'list')

map_osm = folium.Map(location=천안시청, zoom_start=12)

for ix, row in df.iterrows():
    location = (row['lat'], row['lot'])
    folium.Marker(location, popup=row['s_name']).add_to(map_osm)

map_osm